# CosyVoice + vTTS API 테스트 (Kaggle)

**목표**: CosyVoice3 Zero-shot TTS를 vTTS API로 테스트

---

## 사전 요구사항
- Kaggle GPU 활성화 필수! (Settings → Accelerator → GPU T4 x2)
- 예상 시간: 설치 ~15-20분, 테스트 ~5분

## 1. 환경 설정 및 설치 (~15-20분)

In [ ]:
# CosyVoice 설치 (시간이 오래 걸립니다 ~15-20분)
import os
import subprocess
import sys

print("CosyVoice 환경 설정 중...")
print("   ⚠️ 이 과정은 15-20분 정도 소요됩니다.\n")

# 1. numpy + onnxruntime 완전 재설치 (정확한 버전 고정!)
print("   [1/8] numpy + ONNX Runtime 재설치 (CosyVoice 공식 버전)...")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "numpy", "onnxruntime", "onnxruntime-gpu", "-y", "-q"], capture_output=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "numpy==1.26.4", "onnxruntime-gpu==1.18.0"], capture_output=True)

# 2. PyTorch 정확히 고정 (CosyVoice 공식 버전!)
print("   [2/8] PyTorch 설치 (CosyVoice 공식 버전: 2.3.1)...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q",
               "torch==2.3.1", "torchaudio==2.3.1",
               "--index-url", "https://download.pytorch.org/whl/cu121"], capture_output=True)

# 3. vTTS + Supertonic 설치 (참조 오디오 생성용)
print("   [3/8] vTTS + Supertonic 설치 (참조 오디오 생성용)...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "vtts[supertonic] @ git+https://github.com/bellkjtt/vTTS.git", "--no-deps"], capture_output=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "fastapi", "uvicorn", "httpx", "pydantic>=2.0", "pydantic-settings", "loguru", "soundfile", "librosa", "huggingface-hub"], capture_output=True)

# 4. CUDA 지원 확인
print("   [4/8] CUDA 지원 확인...")
import onnxruntime as ort
providers = ort.get_available_providers()
print(f"      ONNX Providers: {providers}")
if "CUDAExecutionProvider" in providers:
    print("      CUDA 지원 활성화!")
else:
    print("      ⚠️ CUDA 미지원 (CPU 모드)")

# 5. CosyVoice 저장소 클론
print("   [5/8] CosyVoice 저장소 클론...")
COSYVOICE_PATH = os.path.expanduser("~/.vtts/CosyVoice")
os.makedirs(os.path.dirname(COSYVOICE_PATH), exist_ok=True)

if not os.path.exists(COSYVOICE_PATH):
    result = subprocess.run(
        ["git", "clone", "--depth", "1", "https://github.com/FunAudioLLM/CosyVoice.git", COSYVOICE_PATH],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        print(f"      CosyVoice 클론 완료: {COSYVOICE_PATH}")
    else:
        print(f"      ❌ 클론 실패: {result.stderr}")
else:
    print(f"      CosyVoice already exists: {COSYVOICE_PATH}")

# 6. CosyVoice 핵심 의존성 먼저 설치 (버전 고정!)
print("   [6/8] CosyVoice 핵심 의존성 설치...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q",
               "transformers==4.51.3",
               "librosa==0.10.2",
               "modelscope==1.20.0",
               "HyperPyYAML==1.2.2",
               "conformer==0.3.2",
               "wetext==0.0.4",
               "x-transformers==2.11.24",
               "diffusers==0.29.0"], capture_output=True)

# 7. CosyVoice 나머지 requirements 설치
print("   [7/8] CosyVoice 나머지 의존성 설치 (오래 걸림)...")
req_file = os.path.join(COSYVOICE_PATH, "requirements.txt")
if os.path.exists(req_file):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", req_file], capture_output=True)
    print("      CosyVoice 의존성 설치 완료!")

# 8. 환경변수 설정
print("   [8/8] 환경변수 설정...")
os.environ["COSYVOICE_PATH"] = COSYVOICE_PATH

print(f"\nCosyVoice 설치 완료!")
print(f"   경로: {COSYVOICE_PATH}")

## 2. 참조 오디오 생성 (Supertonic 사용)

In [ ]:
# 참조 오디오 생성 (CosyVoice Zero-shot 모드에서 사용)
from vtts.engines.supertonic import SupertonicEngine
from vtts.engines.base import TTSRequest
import IPython.display as ipd
from pathlib import Path
import soundfile as sf

print("참조 오디오 생성 중 (Supertonic 사용)...\n")

# Supertonic 엔진 로드
engine = SupertonicEngine(model_id="Supertone/supertonic-2", device="cuda")
engine.load_model()

# 참조 오디오 저장 디렉토리
ref_audio_dir = Path("./reference_audio")
ref_audio_dir.mkdir(exist_ok=True)

# 글로벌 변수로 경로 선언
global ko_path, en_path, zh_path
ko_path, en_path, zh_path = None, None, None

# 한국어 참조 오디오
try:
    request_ko = TTSRequest(
        text="안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다.",
        language="ko",
        voice="F1",
        speed=1.0,
        extra_params={"total_steps": 10, "silence_duration": 0.3}
    )
    output_ko = engine.synthesize(request_ko)
    ko_path = ref_audio_dir / "reference_ko.wav"
    sf.write(ko_path, output_ko.audio, output_ko.sample_rate)
    print(f"한국어 참조 오디오: {ko_path}")
    print(f"   내용: {request_ko.text}\n")
except Exception as e:
    print(f"❌ 한국어 참조 오디오 생성 실패: {e}\n")

# 영어 참조 오디오
try:
    request_en = TTSRequest(
        text="Hello, this is a reference audio for voice cloning.",
        language="en",
        voice="F1",
        speed=1.0,
        extra_params={"total_steps": 10, "silence_duration": 0.3}
    )
    output_en = engine.synthesize(request_en)
    en_path = ref_audio_dir / "reference_en.wav"
    sf.write(en_path, output_en.audio, output_en.sample_rate)
    print(f"영어 참조 오디오: {en_path}")
    print(f"   내용: {request_en.text}\n")
except Exception as e:
    print(f"❌ 영어 참조 오디오 생성 실패: {e}\n")

# 중국어 참조 오디오
try:
    request_zh = TTSRequest(
        text="你好，这是用于语音克隆的参考音频。",
        language="zh",
        voice="F1",
        speed=1.0,
        extra_params={"total_steps": 10, "silence_duration": 0.3}
    )
    output_zh = engine.synthesize(request_zh)
    zh_path = ref_audio_dir / "reference_zh.wav"
    sf.write(zh_path, output_zh.audio, output_zh.sample_rate)
    print(f"중국어 참조 오디오: {zh_path}")
    print(f"   내용: {request_zh.text}\n")
except Exception as e:
    print(f"❌ 중국어 참조 오디오 생성 실패: {e}\n")

# Supertonic 엔진 언로드 (메모리 절약)
engine.unload_model()
print("참조 오디오 생성 완료!\n")

## 3. vTTS 서버 시작 (백그라운드)

In [ ]:
# vTTS 서버 시작 (CosyVoice)
import subprocess
import time
from pathlib import Path

print("vTTS 서버 시작 중...\n")

# 로그 파일
log_file = Path("cosyvoice_server.log")

# 서버 프로세스 시작 (백그라운드)
server_process = subprocess.Popen(
    ["vtts", "serve", "FunAudioLLM/Fun-CosyVoice3-0.5B-2512", "--device", "cuda", "--port", "8000"],
    stdout=open(log_file, "w"),
    stderr=subprocess.STDOUT,
    text=True
)

# 서버 시작 대기 (최대 60초)
print("서버 시작 대기 중 (최대 60초)...")
for i in range(60):
    try:
        import httpx
        response = httpx.get("http://localhost:8000/health", timeout=1.0)
        if response.status_code == 200:
            print(f"서버 시작 완료! ({i+1}초 소요)\n")
            break
    except:
        if i % 5 == 0:
            print(f"   대기 중... ({i+1}/60초)")
        time.sleep(1)
else:
    print("⚠️ 서버 시작 타임아웃. 로그를 확인하세요.\n")

# 최근 로그 출력
print("최근 서버 로그 (마지막 50줄):")
print("=" * 60)
if log_file.exists():
    with open(log_file, "r") as f:
        lines = f.readlines()
        for line in lines[-50:]:
            print(line.rstrip())

## 4. CosyVoice 음성 클로닝 테스트

In [ ]:
# CosyVoice Zero-shot 음성 클로닝 테스트
from vtts import VTTSClient
import IPython.display as ipd

# 클라이언트 생성
client = VTTSClient(base_url="http://localhost:8000")

print("CosyVoice 음성 클로닝 테스트\n" + "=" * 60 + "\n")

# 한국어 테스트 (Zero-shot)
if 'ko_path' in globals() and ko_path and ko_path.exists():
    print("1. 한국어 Zero-shot 음성 클로닝...")
    try:
        audio = client.tts(
            text="안녕하세요, 이것은 CosyVoice를 사용한 음성 클로닝 테스트입니다.",
            model="FunAudioLLM/Fun-CosyVoice3-0.5B-2512",
            language="ko",
            reference_audio=str(ko_path.absolute()),
            reference_text="안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다.",
            speed=1.0
        )
        print(f"   음성 생성 완료!")
        print(f"   Duration: {len(audio.audio)/audio.sample_rate:.2f}s")
        print(f"   Sample rate: {audio.sample_rate}Hz\n")
        
        # 재생
        display(ipd.Audio(audio.audio, rate=audio.sample_rate))
        
        # 저장
        audio.save("cosyvoice_ko_test.wav")
        print("   Saved: cosyvoice_ko_test.wav\n")
    except Exception as e:
        print(f"   ❌ 실패: {e}\n")
else:
    print("1. ⚠️ 한국어 참조 오디오 없음 (건너뜀)\n")

# 영어 테스트 (Zero-shot)
if 'en_path' in globals() and en_path and en_path.exists():
    print("2. 영어 Zero-shot 음성 클로닝...")
    try:
        audio = client.tts(
            text="Hello, this is a voice cloning test using CosyVoice.",
            model="FunAudioLLM/Fun-CosyVoice3-0.5B-2512",
            language="en",
            reference_audio=str(en_path.absolute()),
            reference_text="Hello, this is a reference audio for voice cloning.",
            speed=1.0
        )
        print(f"   음성 생성 완료!")
        print(f"   Duration: {len(audio.audio)/audio.sample_rate:.2f}s")
        print(f"   Sample rate: {audio.sample_rate}Hz\n")
        
        # 재생
        display(ipd.Audio(audio.audio, rate=audio.sample_rate))
        
        # 저장
        audio.save("cosyvoice_en_test.wav")
        print("   Saved: cosyvoice_en_test.wav\n")
    except Exception as e:
        print(f"   ❌ 실패: {e}\n")
else:
    print("2. ⚠️ 영어 참조 오디오 없음 (건너뜀)\n")

# 중국어 테스트 (Zero-shot)
if 'zh_path' in globals() and zh_path and zh_path.exists():
    print("3. 중국어 Zero-shot 음성 클로닝...")
    try:
        audio = client.tts(
            text="你好，这是使用CosyVoice进行语音克隆的测试。",
            model="FunAudioLLM/Fun-CosyVoice3-0.5B-2512",
            language="zh",
            reference_audio=str(zh_path.absolute()),
            reference_text="你好，这是用于语音克隆的参考音频。",
            speed=1.0
        )
        print(f"   음성 생성 완료!")
        print(f"   Duration: {len(audio.audio)/audio.sample_rate:.2f}s")
        print(f"   Sample rate: {audio.sample_rate}Hz\n")
        
        # 재생
        display(ipd.Audio(audio.audio, rate=audio.sample_rate))
        
        # 저장
        audio.save("cosyvoice_zh_test.wav")
        print("   Saved: cosyvoice_zh_test.wav\n")
    except Exception as e:
        print(f"   ❌ 실패: {e}\n")
else:
    print("3. ⚠️ 중국어 참조 오디오 없음 (건너뜀)\n")

print("테스트 완료!")

## 5. HTTP API 직접 호출 테스트

In [ ]:
# HTTP API 직접 호출
import httpx
import json
import IPython.display as ipd

print("HTTP API 직접 호출 테스트\n" + "=" * 60 + "\n")

# 참조 오디오 Base64 인코딩
import base64

if 'ko_path' in globals() and ko_path and ko_path.exists():
    with open(ko_path, "rb") as f:
        ko_audio_b64 = base64.b64encode(f.read()).decode()
    
    # TTS API 호출 (Zero-shot)
    tts_payload = {
        "model": "FunAudioLLM/Fun-CosyVoice3-0.5B-2512",
        "input": "이것은 HTTP API를 통한 CosyVoice 테스트입니다.",
        "language": "ko",
        "reference_audio": ko_audio_b64,
        "reference_text": "안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다.",
        "response_format": "wav",
        "speed": 1.0
    }
    
    response = httpx.post(
        "http://localhost:8000/v1/audio/speech",
        json=tts_payload,
        timeout=60.0
    )
    
    if response.status_code == 200:
        with open("http_api_test.wav", "wb") as f:
            f.write(response.content)
        print("HTTP API 호출 성공!")
        print(f"   Status: {response.status_code}")
        print(f"   Content-Type: {response.headers.get('content-type')}")
        print(f"   Size: {len(response.content)} bytes")
        print("   Saved: http_api_test.wav\n")
        
        # 재생
        display(ipd.Audio(response.content, rate=22050))
    else:
        print(f"❌ Error: {response.status_code}")
        print(response.text)
else:
    print("⚠️ 한국어 참조 오디오 없음 (건너뜀)")

## 6. 정리 (서버 종료)

In [ ]:
# 서버 종료
if 'server_process' in globals():
    print("vTTS 서버 종료 중...\n")
    server_process.terminate()
    server_process.wait()
    print("서버 종료 완료!\n")
else:
    print("⚠️ 서버 프로세스를 찾을 수 없습니다.\n")

print("=" * 60)
print("테스트 완료!")
print("=" * 60)